In [4]:
using Pkg
Pkg.activate(homedir() * "/JuliaEnvs/numerics")
using ApproxFun, Polynomials


  Activating project at `~/JuliaEnvs/numerics`


In [11]:
using LinearAlgebra
using Plots

# 设置初始条件和边界条件
L = 1.0 # 区间长度
N = 100 # 离散化点数
t_end = 0.1 # 时间终点
dt = 0.0001 # 时间步长

x = range(-1, 1, length=N+2)[2:end-1] # 离散化空间点
T = zeros(N) # 初始温度
bc = zeros(N) # 边界条件

# 设置初始条件和边界条件
T .= sin.(π * x)
bc .= 0.0

# 定义Chebyshev多项式及其求导函数
function Tn(n::Int, x::Vector{T}) where T
    if n == 0
        return ones(T, length(x))
    elseif n == 1
        return x
    else
        return 2 * x .* Tn(n-1, x) .- Tn(n-2, x)
    end
end

function dTn(n::Int, x::Vector{T}) where T
    return n * (Tn(n-1, x) .+ x .* dTn(n-1, x)) ./ (1 .- x.^2)
end

# 计算Chebyshev-Galerkin矩阵
function compute_A(N::Int)
    A = zeros(N, N)
    for n = 1:N, m = 1:N
        if n == m
            A[n, m] = 2 / (1 - x[n]^2)
        else
            A[n, m] = (Tn(n, x) .* dTn(m, x)) |> sum
        end
    end
    return A
end

A = compute_A(N)

# 显示初始温度分布
plot(x, T, label="t=0", xlabel="x", ylabel="T(x)")

# 使用显式欧拉方法求解时间演化
for t = dt:dt:t_end
    T += dt * A * T
    T[1] = bc[1] # 左边界
    T[end] = bc[end] # 右边界

    # 显示温度随时间的变化
    if t == 0.01 || t == 0.05 || t == 0.1
        plot!(x, T, label="t=$t")
    end
end

# 显示最终温度分布
plot!(x, T, label="t=$t_end", legend=:bottomright)


LoadError: invalid redefinition of constant bc